# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!


### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [2]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database '\\dc1\userspace\kummer\bosssdbs-minibatch\OscillatingDroplet3D'.


In [4]:
OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\fdygitrunner\ValidationTests\OscillatingDroplet3D'.


In [5]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm1_mode4_aP2*	11/01/2021 20:00:02	c025e47c...
#1: OscillatingDroplet3D	J432k3_arm1_mode2_aP1*	11/01/2021 19:36:06	c5c88e22...
#2: OscillatingDroplet3D	J432k3_arm1_mode2_aP0*	11/01/2021 19:36:06	ece5c538...
#3: OscillatingDroplet3D	J432k3_arm1_mode2_aP2*	11/01/2021 19:36:06	c2c08313...
#4: OscillatingDroplet3D	J432k3_arm1_mode4_aP0*	11/01/2021 19:36:07	bc918d99...
#5: OscillatingDroplet3D	J432k3_arm1_mode4_aP1*	11/01/2021 19:36:06	4e3d2799...
#6: OscillatingDroplet3D	J432k3_arm1_mode3_aP2*	11/01/2021 19:36:06	925309b2...
#7: OscillatingDroplet3D	J432k3_arm1_mode3_aP1*	11/01/2021 19:36:06	e78dc9ef...
#8: OscillatingDroplet3D	J432k3_arm1_mode3_aP0*	11/01/2021 19:36:06	370651e3...
#9: OscillatingDroplet3D	J432k3_arm0_mode4_aP2*	11/01/2021 19:30:57	209fe1bd...
#10: OscillatingDroplet3D	J432k3_arm0_mode4_aP1*	11/01/2021 19:24:22	6f496029...
#11: OscillatingDroplet3D	J432k3_arm0_mode4_aP0*	11/01/2021 19:24:04	8525c74a...
#12: OscillatingDroplet3D	J432k3_arm0_

In [6]:
//var S1 = wmg.Sessions[0];

In [7]:
//S1.Tags 

In [8]:
//foreach(var kv in S1.KeysAndQueries)
//   Console.WriteLine($"{kv.Key} \t\t= {kv.Value}");

In [9]:
using System.IO;

In [16]:
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}

In [19]:
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}

sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\c025e47c-8fcc-4964-a39c-65692d984d55\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\c5c88e22-824c-4821-beb3-18ac149a33f4\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\ece5c538-288a-4a9c-b6cb-9622fe7d292d\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\c2c08313-67cb-4886-8439-4857fe3bde72\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\bc918d99-b1c9-4972-9837-280d189367f9\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\4e3d2799-7f09-43ab-a5e0-a5db293774c6\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\925309b2-4f95-47b1-8265-4b16dbbf9302\SphericalHarmonics.txt
sanitizing: \\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessi

In [20]:
//FileSanitizer(@"\\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\c5c88e22-824c-4821-beb3-18ac149a33f4\SphericalHarmonics.txt");

In [22]:
//S1.PrintSessionDirectory();

In [23]:
//var tab = S1.ReadTabulatedTextFileAsDoubles("SphericalHarmonics-Sanitized.txt", '\t');

In [25]:
/*var plot = new Plot2Ddata();
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var time = tab["time"];
int cnt = -1;
foreach(var column in tab) {
   cnt++;
   var fmt = new PlotFormat();
   fmt.Style = Styles.Lines; 
   fmt.LineColor = allColors[cnt%allColors.Length];
   if(column.Key == "time")
      continue;
   plot.AddDataGroup(column.Key, time, column.Value, fmt);
   
   plot.Title = S1.Name;
}*/

In [27]:
//plot.PlotNow()

In [28]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm1_mode4_aP2*	11/01/2021 20:00:02	c025e47c...
#1: OscillatingDroplet3D	J432k3_arm1_mode2_aP1*	11/01/2021 19:36:06	c5c88e22...
#2: OscillatingDroplet3D	J432k3_arm1_mode2_aP0*	11/01/2021 19:36:06	ece5c538...
#3: OscillatingDroplet3D	J432k3_arm1_mode2_aP2*	11/01/2021 19:36:06	c2c08313...
#4: OscillatingDroplet3D	J432k3_arm1_mode4_aP0*	11/01/2021 19:36:07	bc918d99...
#5: OscillatingDroplet3D	J432k3_arm1_mode4_aP1*	11/01/2021 19:36:06	4e3d2799...
#6: OscillatingDroplet3D	J432k3_arm1_mode3_aP2*	11/01/2021 19:36:06	925309b2...
#7: OscillatingDroplet3D	J432k3_arm1_mode3_aP1*	11/01/2021 19:36:06	e78dc9ef...
#8: OscillatingDroplet3D	J432k3_arm1_mode3_aP0*	11/01/2021 19:36:06	370651e3...
#9: OscillatingDroplet3D	J432k3_arm0_mode4_aP2*	11/01/2021 19:30:57	209fe1bd...
#10: OscillatingDroplet3D	J432k3_arm0_mode4_aP1*	11/01/2021 19:24:22	6f496029...
#11: OscillatingDroplet3D	J432k3_arm0_mode4_aP0*	11/01/2021 19:24:04	8525c74a...
#12: OscillatingDroplet3D	J432k3_arm0_

In [29]:
string[] modes = new string[]{ "mode2", "mode3", "mode4"};
string[] aPis = new string[] { "aP0", "aP1", "aP2" };
string[] amrS = new string[] { "arm0", "arm1" };

In [60]:
Plot2Ddata[,] PlotTable = new Plot2Ddata[3,3];
for(int iCol = 0; iCol < 3; iCol++) {
for(int iRow = 0; iRow < 3; iRow++) {
for(int iarm = 0; iarm < 2; iarm++) {
    string _mode = modes[iRow];
    string _aP = aPis[iCol];
    string _amr = amrS[iarm];

    ISessionInfo SI = wmg.Sessions.Single(sess => sess.Name.Contains(_mode) 
                                               && sess.Name.Contains(_aP) 
                                               && sess.Name.Contains(_amr));
    Console.WriteLine(SI.Name);

    var tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics-Sanitized.txt", '\t');
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var column in tab) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
       if(column.Key == "time")
          continue;
       if(iarm > 0)
          fmt.DashType = DashTypes.Solid;
       else 
          fmt.DashType = DashTypes.Dashed;
       string name = column.Key;
       if(iarm > 0)
           name = name + "-amr";
       plot.AddDataGroup(name, time, column.Value, fmt);
   }
   
   plot.ShowLegend = iCol == 2 && iRow == 2;
   
   plot.Title = SI.Name;
   
   if(iarm == 0) {
       PlotTable[iRow,iCol] = plot;
   } else {
       PlotTable[iRow,iCol] = PlotTable[iRow,iCol].Merge(plot);
   }
}
PlotTable[iRow,iCol].ShowLegend = iCol == 2 && iRow == 2;
if(iRow == 0)
   PlotTable[iRow,iCol].Title = aPis[iCol];
if(iCol == 0)
   PlotTable[iRow,iCol].Ylabel = modes[iRow];
}
}


J432k3_arm0_mode2_aP0
J432k3_arm1_mode2_aP0
J432k3_arm0_mode3_aP0
J432k3_arm1_mode3_aP0
J432k3_arm0_mode4_aP0
J432k3_arm1_mode4_aP0
J432k3_arm0_mode2_aP1
J432k3_arm1_mode2_aP1
J432k3_arm0_mode3_aP1
J432k3_arm1_mode3_aP1
J432k3_arm0_mode4_aP1
J432k3_arm1_mode4_aP1
J432k3_arm0_mode2_aP2
J432k3_arm1_mode2_aP2
J432k3_arm0_mode3_aP2
J432k3_arm1_mode3_aP2
J432k3_arm0_mode4_aP2
J432k3_arm1_mode4_aP2


In [61]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Program Files\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 



(2,1): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.003 
 
 
 
 
 -0.002 
 
 
 
 
 -0.001 
 
 
 
 
 0 
 
 
 
 
 0.001 
 
 
 
 
 0.002 
 
 
 
 
 0.003 
 
 
 
 
 0.004 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mode2 
 
 
 
 
 aP0 
 
 
 (0, 0) 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='2.5,4.0' d='M94.5,116.7 L95.4,116.2 L96.3,116.0 L97.2,115.9 L98.1,116.1 L99.0,116.4 L99.9,116.8 L100.8,117.3
 L101.7,117.9 L102.6,118.6 L103.5,119.4 L104.4,120.3 L105.3,121.2 L106.2,122.3 L107.1,123.4 L108.0,124.6
 L108.9,125.9 L109.8,127.3 L110.7,128.7 L111.6,130.1 L112.5,131.7 L113.4,133.3 L114.3,134.9 L115.2,136.6
 L116.1,138.3 L117.0,140.1 L117.9,141.9 L118.8,143.8 L119.7,145.7 L120.6,147.6 L121.5,149.5 L122.4,151.5
 L123.3,153.4 L124.2,155.4 L125.1,157.4 L126.0,159.4 L126.9,161.4 L127.8,163.3 L128.7,165.3 L129.6,167.3
 L130.5,169.3 L131.4,171.2 L132.3,173.2 L133.2,175.1 L134.1,177.0 L135.0,178.8 L135.9,180.7 L136.8,182.5
 L137.7,184.2 L138.6,186.0 L139.5,187.6 L140.4,189.3 L141.3,190.9 L142.2,192.5 L143.1,194.0 L144.0,195.4
 L144.9,196.9 L145.8,198.2 L146.7,199.5 L147.6,200.8 L148.5,202.0 L149.4,203.1 L150.3,204.2 L151.1,205.2
 L152.0,206.2 L152.9,207.1 L153.8,208.0 L154.7,208.8 L155.6,209.5 L156.5,210.1 L157.4,210.7 L158.3,211.3
 L159.2,211.8 L160.1,212.2 L161.0,212.5 L161.9,212.8 L162.8,213.1 L163.7,213.2 L164.6,213.4 L165.5,213.4
 L166.4,213.4 L167.3,213.4 L168.2,213.3 L169.1,213.1 L170.0,212.9 L170.9,212.6 L171.8,212.3 L172.7,211.9
 L173.6,211.5 L174.5,211.0 L175.4,210.5 L176.3,210.0 L177.2,209.4 L178.1,208.8 L179.0,208.1 L179.9,207.4
 L180.8,206.6 L181.7,205.8 L182.6,205.0 L183.5,204.2 L184.4,203.3 L185.3,202.4 L186.2,201.5 L187.1,200.5
 L188.0,199.5 L188.9,198.5 L189.8,197.5 L190.7,196.5 L191.6,195.4 L192.5,194.3 L193.4,193.3 L194.3,192.2
 L195.2,191.1 L196.1,189.9 L197.0,188.8 L197.9,187.7 L198.8,186.6 L199.7,185.4 L200.6,184.3 L201.5,183.2
 L202.4,182.0 L203.3,180.9 L204.2,179.8 L205.1,178.7 L206.0,177.5 L206.9,176.4 L207.8,175.3 L208.7,174.3
 L209.6,173.2 L210.5,172.1 L211.4,171.1 L212.3,170.0 L213.2,169.0 L214.1,168.0 L215.0,167.0 L215.9,166.1
 L216.8,165.1 L217.7,164.2 L218.6,163.2 L219.5,162.3 L220.4,161.5 L221.3,160.6 L222.2,159.8 L223.1,158.9
 L224.0,158.1 L224.9,157.3 L225.8,156.6 L226.7,155.8 L227.6,155.1 L228.5,154.4 L229.4,153.7 L230.3,153.0
 L231.2,152.3 L232.1,151.7 L233.0,151.0 L233.9,150.4 L234.8,149.8 L235.7,149.2 L236.6,148.6 L237.5,148.1
 L238.4,147.5 L239.3,147.0 L240.2,146.5 L241.1,145.9 L242.0,145.4 L242.9,144.9 L243.8,144.4 L244.7,143.9
 L245.6,143.4 L246.5,143.0 L247.4,142.5 L248.3,142.0 L249.2,141.5 L250.1,141.1 L251.0,140.6 L251.9,140.1
 L252.8,139.6 L253.7,139.2 L254.6,138.7 L255.5,138.2 L256.4,137.7 L257.3,137.2 L258.2,139.0 L259.1,137.3
 L260.0,136.2 L260.9,135.5 L261.8,134.9 L262.7,134.3 L263.5,133.8 L264.4,133.2 L265.3,132.7 L266.2,132.1
 L267.1,131.6 L268.0,131.0 L268.9,130.5 L269.8,129.9 L270.7,129.3 L271.6,128.8 L272.5,128.2 L273.4,127.6
 L274.3,127.0 L275.2,126.5 L276.1,125.9 L277.0,125.3 L277.9,124.7 L278.8,124.1 L279.7,123.5 L280.6,123.0
 L281.5,122.4 L282.4,121.8 L283.3,121.3 L284.2,120.7 L285.1,120.2 L286.0,119.7 L286.9,119.2 L287.8,120.7
 L288.7,119.9 L289.6,118.8 L290.5,119.3 L291.4,118.2 L292.3,117.4 L293.2,116.7 L294.1,116.1 L295.0,115.7
 L295.9,115.2 L296.8,114.8 L297.7,114.5 L298.6,114.1 L299.5,113.7 L300.4,113.4 L301.3,113.0 L302.2,112.6
 L303.1,112.2 L304.0,111.7 L304.9,111.2 L305.8,110.6 L306.7,109.8 L307.6,109.1 L308.5,110.0 L309.4,108.6
 L310.3,107.6 L311.2,106.6 L312.1,105.8 L313.0,105.0 L313.9,104.3 L314.8,103.6 L315.7,103.0 L316.6,102.5
 L317.5,102.0 L

In [62]:
var cl = gp.PlotCairolatex(xSize:32,ySize:18);
cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [64]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}